[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dimitrisdais/generative-ai-lab/blob/main/notebooks/blog_to_video_ai_generator.ipynb)

# 🎬 AI Blog-to-Video Generator

👋 Hi, I'm **Dimitris Dais**, an engineer passionate about AI creativity tools.

This notebook turns a blog post into a complete narrated video using:

Step 1: Generate the Blog Structure with an LLM  
Step 2: Write Paragraphs for Each Section with an LLM  
Step 3: Create Visuals for Each Section with a Text-to-Image Model  
Step 4: Generate Audio Narration with a Text-to-Speech Model  
Step 5: Combine Frames and Audio into the Final Video  

For more explanation, refer to the [corresponding blog](https://dimitrisdais.github.io/dimitris-dais.github.io/ai/multimodal/blog_to_video_ai_generator/).

Enjoyed it? Reuse or expand it — and feel free to connect.

🔗 **Website**: [dimitrisdais.github.io](https://dimitrisdais.github.io/dimitris-dais.github.io/)  
📬 **Contact**: dimitris.dais.phd@gmail.com  
🐙 **GitHub**: [@dimitrisdais](https://github.com/dimitrisdais)  
🔗 **LinkedIn**: [linkedin.com/in/dimitris-dais](https://www.linkedin.com/in/dimitris-dais/)  
▶️ **YouTube**: [youtube.com/@dimitrisdais](https://www.youtube.com/channel/UCuSdAarhISVQzV2GhxaErsg)

### 🔗 Mount Google Drive

This step connects your Google Drive to the Colab notebook so that we can save and access files like generated images, audio, and videos.

➡️ A pop-up will appear asking you to **authorize** access — click the link, choose your Google account, and copy the authorization code back into the notebook when prompted

![Mount Google Drive](https://raw.githubusercontent.com/dimitrisdais/generative-ai-lab/main/assets/images/mount_google_drive.png)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 🔧 Install Required Packages

In [ ]:
!pip install -U TTS transformers==4.46.2 accelerate==1.1.1 bitsandbytes==0.45.2 datasets==3.1.0 huggingface-hub==0.26.2 safetensors==0.4.5 -q

In [ ]:
!apt-get update && apt-get install -y espeak libespeak1

In [ ]:
import ctypes; ctypes.cdll.LoadLibrary("libespeak.so.1")

### ⚙️ Check for GPU Availability

This step checks whether a GPU is available in the current Colab environment and assigns the appropriate device (`"cuda"` for GPU or `"cpu"` otherwise).  
Using a GPU can significantly speed up model inference for image and audio generation.


In [ ]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device", device)

### 📁 Create Output Folders

In [ ]:
import os
from datetime import datetime

base_dir = "/content/blog_ai_video"
timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
session_dir = os.path.join(base_dir, timestamp)
audio_dir = os.path.join(session_dir, "audio")
frames_dir = os.path.join(session_dir, "frames")
video_dir = os.path.join(session_dir, "video")

for d in [audio_dir, frames_dir, video_dir]:
    os.makedirs(d, exist_ok=True)

print("Output directory created:", session_dir)

## 💡 Step 1: Generate the Blog Structure with an LLM

### 🔐 Authenticate with Hugging Face Hub

To access certain models (especially large or gated ones), you may need to log in to your Hugging Face account.

This step will prompt you to enter your **Hugging Face access token**, which you can obtain from [your account's tokens page](https://huggingface.co/settings/tokens).  
Once authenticated, the notebook will have permission to download and use models that require login.

![Hugging Face - Login](https://raw.githubusercontent.com/dimitrisdais/generative-ai-lab/main/assets/images/hugging_face_login.png)  

![Hugging Face - Access Repository](https://raw.githubusercontent.com/dimitrisdais/generative-ai-lab/main/assets/images/hugging_face_access_repository.png)  

In [ ]:
from huggingface_hub import login

login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, BitsAndBytesConfig
import re

In [ ]:
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="float16"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=bnb_config
)

generator = pipeline("text-generation", model=model, tokenizer=tokenizer)

In [ ]:
# Generate blog title
title_prompt = "Suggest one short and funny blog post title. Avoid using listicles or numbers like '10 ways'."

# Generate with sampling
output = generator(
    title_prompt,
    max_new_tokens=50,
    do_sample=True,
    top_k=50,
    temperature=0.9
)

# Extract clean title
blog_title = output[0]["generated_text"].replace(title_prompt, "").strip()
del output

# Print result
print("📘 Blog Title:\n", blog_title)

In [ ]:
# Generate section titles
section_prompt = f"### Instruction:\nWrite five short and creative blog section titles for a blog titled: '{blog_title}'. Format them as a numbered list.\n### Response:"

section_output = generator(
    section_prompt,
    max_new_tokens=150,
    do_sample=True,
    temperature=0.9
)[0]["generated_text"]

section_titles = re.findall(r"\d+\.\s*(.*?)(?=\n\d+\.|$)", section_output.strip())
print("📑 Sections:\n")
for t in section_titles: print("-", t)

## ✍️ Step 2: Write Paragraphs for Each Section with an LLM

In [ ]:
blog = {}

for title in section_titles:

    prompt = f"### Instruction:\nWrite a short paragraph (2-3 sentences) for a blog section titled: '{title}'.\n### Response:"

    response = generator(prompt, max_new_tokens=200, do_sample=True, temperature=0.9)[0]["generated_text"]
    blog[title] = response.replace(prompt, "").strip()
    print(f"📝 {title}:\n", blog[title], "\n")

## 🎨 Step 3: Create Visuals for Each Section with a Text-to-Image Model

This step uses a Stable Diffusion model to generate images for each blog section. For every section title, the model creates a relevant visual based on the provided text prompt.


In [ ]:
from diffusers import StableDiffusionPipeline
from PIL import Image

In [ ]:
sd_pipe = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4",
    torch_dtype=torch.float32
)
sd_pipe = sd_pipe.to("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
image_paths = []

for i, title in enumerate(blog.keys()):

    prompt = f"An eye-catching illustration representing: {title}"

    print(f"Generating image {i+1} for: '{title}'")

    image = sd_pipe(prompt).images[0]
    path = f"{frames_dir}/frame_{i:02d}.png"
    image.save(path)
    image_paths.append(path)

In [ ]:
# Show images
from IPython.display import display
for p in image_paths:
    display(Image.open(p))

## 🔊 Step 4: Generate Audio Narration with a Text-to-Speech Model

In [ ]:
from TTS.api import TTS

In [ ]:
tts = TTS(
    model_name="tts_models/en/vctk/vits",
    progress_bar=False,
).to(device)

In [ ]:
audio_paths = []

for i, (title, text) in enumerate(blog.items()):
    audio_path = os.path.join(audio_dir, f"section_{i:02d}.wav")
    tts.tts_to_file(text=text, file_path=audio_path, speaker="p225")
    audio_paths.append(audio_path)

In [ ]:
from IPython.display import Audio
for path in audio_paths:
    display(Audio(path))

## 🎬 Step 5: Combine Frames and Audio into the Final Video

In [ ]:
import subprocess
import os

for i in range(5):  # Adjust loop if needed
    frame_path = os.path.join(frames_dir, f"frame_{i:02d}.png")
    audio_path = os.path.join(audio_dir, f"section_{i:02d}.wav")
    out_video_path = os.path.join(video_dir, f"clip_{i:02d}.mp4")

    print(f"🔧 Generating video {i + 1}/5")
    print(f"📤 Output path: {out_video_path}")

    cmd = [
        "ffmpeg", "-y",
        "-loop", "1", "-i", frame_path,
        "-i", audio_path,
        "-c:v", "libx264", "-tune", "stillimage",
        "-shortest", "-vf", "scale=1280:720",
        "-pix_fmt", "yuv420p",
        out_video_path
    ]

    try:
        subprocess.run(cmd, check=True)
        print("✅ Video created successfully.\n")
    except subprocess.CalledProcessError as e:
        print(f"❌ Failed to generate video {i}: {e}\n")

In [ ]:
# Create a text file that lists all video parts
with open(os.path.join(video_dir, "file_list.txt"), "w") as f:
    for i in range(5):
        f.write(f"file 'clip_{i:02d}.mp4'\n")

In [ ]:
# Paths to your input list file and output video
video_list_path = os.path.join(video_dir, "file_list.txt")
video_output_path = os.path.join(session_dir, "final_blog_video.mp4")

print(f"📋 Concatenating videos listed in: {video_list_path}")
print(f"📤 Final output path: {video_output_path}")

# FFmpeg concat command using subprocess to avoid UTF-8 locale error in Colab
cmd = [
    "ffmpeg", "-y",
    "-f", "concat", "-safe", "0",
    "-i", video_list_path,
    "-c", "copy",
    video_output_path
]

try:
    subprocess.run(cmd, check=True)
    print("✅ Final video assembled successfully!")
except subprocess.CalledProcessError as e:
    print(f"❌ FFmpeg failed: {e}")


### Copy Output to Google Drive

In [ ]:
import shutil

drive_path = "/content/drive/MyDrive/blog_ai_video_output"
dest_path = os.path.join(drive_path, timestamp)

shutil.copytree(session_dir, dest_path, dirs_exist_ok=True)

print(f"✅ Copied everything to Google Drive at: {dest_path}")